<a href="https://colab.research.google.com/github/quadribello/Hamoye/blob/master/Quadri_Regression_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#path of dataset
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv'
df = pd.read_csv(path)
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
# removing the date, lights columns according to the instruction
df = df.drop(['date','lights'], axis=1)
df

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,55.200000,7.026667,84.256667,17.200000,41.626667,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,60,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,55.200000,6.833333,84.063333,17.200000,41.560000,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,50,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,55.090000,6.560000,83.156667,17.200000,41.433333,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,50,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,55.090000,6.433333,83.423333,17.133333,41.290000,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,60,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,55.090000,6.366667,84.893333,17.200000,41.230000,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,52.400000,24.796667,1.000000,24.500000,44.500000,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,90,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,52.326667,24.196667,1.000000,24.557143,44.414286,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,270,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,52.266667,23.626667,1.000000,24.540000,44.400000,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,420,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,52.200000,22.433333,1.000000,24.500000,44.295714,24.6625,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


In [4]:
#Firstly, we normalise our dataset to a common scale using the min max scaler 
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
normalised_df = pd.DataFrame(scalar.fit_transform(df), columns=df.columns)
features_df = normalised_df.drop(columns=['Appliances'])
target_variable = normalised_df['Appliances']

In [5]:
#Now, we split our dataset into the training and testing dataset.
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(features_df, target_variable, test_size=0.3, random_state=42)
print("number of test samples :", X_val.shape[0])
print("number of training samples:",X_train.shape[0])
#Define model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(X_train,y_train)
#obtain predictions
predicted_values = linear_model.predict(X_val)
# R squraed on the dataset
linear_model.score(X_val,y_val)

number of test samples : 5921
number of training samples: 13814


0.14890246319303535

In [8]:
# Question 12
##From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) 
##and the temperature outside the building (y = T6). What is the R^2 value in two D.P?
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(df[['T2']],df[['T6']])
r2=lm.score(df[['T2']],df[['T6']])
print('r2_score = ',r2.round(2))

r2_score =  0.64


In [17]:
# Question 13
## What is the Mean Absolute Error (in two decimal places)?
from sklearn.metrics import mean_absolute_error
mse = mean_absolute_error(y_val,predicted_values)
print('The mean square error is: ', mse.round(2))

The mean square error is:  0.05


In [15]:
# Question 14
## What is the Residual Sum of Squares (in two decimal places)?
rss = np.sum(np.square(y_val - predicted_values))
round(rss, 2) # rounds the rss value to 2 dp

45.35

In [19]:
# Question 15
## What is the Root Mean Squared Error (in three decimal places)?
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_val, predicted_values))
print('The root mean square error  is: ', rmse.round(3))

The root mean square error  is:  0.088


In [23]:
# Question 16
## What is the Coefficient of Determination (in two decimal places)? 
from sklearn.metrics import r2_score
r2_score = r2_score(y_val, predicted_values)
round(r2_score, 2) # rounds the r-square value to 2 dp

0.15

In [39]:
# Question 17
## Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001, random_state=2)
lasso_reg.fit(X_train,y_train)
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features',col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(linear_model, X_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_Weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on ='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_Weight
0,RH_2,-0.456698,-0.411071,-0.000000
1,T_out,-0.321860,-0.262172,0.000000
2,T2,-0.236178,-0.201397,0.000000
3,T9,-0.189941,-0.188916,-0.000000
4,RH_8,-0.157595,-0.156830,-0.000110
5,RH_out,-0.077671,-0.054724,-0.049557
6,RH_7,-0.044614,-0.045977,-0.000000
7,RH_9,-0.039800,-0.041367,-0.000000
8,T5,-0.015657,-0.019853,-0.000000
9,T1,-0.003281,-0.018406,0.000000


In [41]:
# Question 18
##Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4, random_state=2)
ridge_reg.fit(X_train,y_train)
rmse_ridge = np.sqrt(mean_squared_error(y_val, predicted_values))
print(round(rmse_ridge, 3))

0.088


In [53]:
# Question 19
## Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001, random_state=2)
lasso_reg.fit(X_train,y_train)
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features',col_name]
  weights_df[col_name].round(3)
  return weights_df


lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_Weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on ='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_Weight
0,RH_2,-0.456698,-0.411071,-0.000000
1,T_out,-0.321860,-0.262172,0.000000
2,T2,-0.236178,-0.201397,0.000000
3,T9,-0.189941,-0.188916,-0.000000
4,RH_8,-0.157595,-0.156830,-0.000110
5,RH_out,-0.077671,-0.054724,-0.049557
6,RH_7,-0.044614,-0.045977,-0.000000
7,RH_9,-0.039800,-0.041367,-0.000000
8,T5,-0.015657,-0.019853,-0.000000
9,T1,-0.003281,-0.018406,0.000000


In [50]:
# Question 20
y_pred_lasso = lasso_reg.predict(X_val)
rmse=np.sqrt(mean_squared_error(y_val,y_pred_lasso))
print('rmse =',rmse.round(3))

rmse = 0.094
